In [1]:
import sys
sys.path.append('../')
import numpy as np

In [2]:
from mfGPR.models import GPRModel, GPRModel_multiFidelity

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [3]:
def high(x):
    x1 = x[:,0]
    x2 = x[:,1]
    return (-1.275*x1**2 / np.pi**2 + 5.0*x1/np.pi + x2 - 6.0)**2 + (10.0 - 5.0/(4.0*np.pi))*np.cos(x1) + 10.0

def medium(x):
    x1 = x[:,0]
    x2 = x[:,1]
    return 10.0*np.sqrt(high(x-2.0)) + 2.0*(x1-0.5)-3.0*(3.0*x2-1.0) - 1.0

def low(x):
    x1 = x[:,0]
    x2 = x[:,1]
    return medium(1.2*(x+2.0)) - 3.0*x2 + 1.0
def scale_range(x,ub,lb):
    Np = x.shape[0]
    dim = x.shape[1]
    for i in range(0,Np):
        for j in range(0,dim):
            tmp = ub[j] -lb[j]
            x[i][j] = tmp*x[i][j] + lb[j]
    return x

In [4]:
N1 = 80
N2 = 40
N3 = 20

plot = 1
save = 0

dim = 2
lb = np.array([-5.0, 0.0])
ub = np.array([10.0, 15.0])

tmp = np.random.rand(1000,dim)
Xtrain = scale_range(tmp,ub,lb)
idx = np.random.permutation(1000)
X1 = Xtrain[idx[0:N1], :]
X2 = Xtrain[idx[0:N2], :]
X3 = Xtrain[idx[0:N3], :]

Y1 = low(X1)[:,None]
Y2 = medium(X2)[:,None]
Y3 = high(X3)[:,None]



In [5]:
m1 = GPRModel(X1, Y1)

 /project2/andrewferguson/Kirill/env_mfGPR/lib/python3.7/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /project2/andrewferguson/Kirill/env_mfGPR/lib/python3.7/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /project2/andrewferguson/Kirill/env_mfGPR/lib/python3.7/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /project2/andrewferguson/Kirill/env_mfGPR/lib/python3.7/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


In [6]:
m1_1 = GPRModel(X1, Y1,)

In [7]:
m2 = GPRModel_multiFidelity(X2, Y2, m1)

In [28]:
m2.predict(X2, return_samples=True).shape

(100, 40)

In [8]:
m2_1 = GPRModel_multiFidelity(X2, Y2, [m1, m1_1], theta=[0.5, 0.5])

In [12]:
m3 = GPRModel_multiFidelity(X3, Y3, [m2, m2_1], theta=[0.5, 0.5])

In [30]:
m3 = GPRModel_multiFidelity(X3, Y3, [m2, m1], theta=[0.5, 0.5])

In [22]:
m2.n_samples

10

In [10]:
m3 = GPRModel_multiFidelity(X3, Y3, [m2, m1], theta=[0.5, 0.5])

In [11]:
m3 = GPRModel_multiFidelity(X3, Y3, m2)

In [29]:
m3.predict(X3, return_samples=True).shape

(1000, 20)

In [11]:
10 ** 3

1000

In [9]:
m2.predict(X3)[0].shape

(20, 1)

In [10]:
X3.shape

(20, 2)